In [20]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LSTM, TimeDistributed, Embedding, Flatten, Activation, Dot, Concatenate

In [21]:


data = pd.read_csv(r"C:\Users\Owner\Downloads\MRP Project\CPI\CPI_Monthly_2003_2023.csv", parse_dates=[0],index_col=[0]
                  )
data.head()


,All-items,Food,Shelter,Household operations,Clothing and footwear,Transportation,Gasoline,Health and personal care,Recreation and education,"Alcohol, tobacco and cannabis",All-items excluding food and energy,All-items excluding energy,Energy,Goods,Services
Date,,,,,,,,,,,,,,,
2003-01-01,102.0,101.6,102.1,100.1,96.6,105.5,108.4,100.5,99.3,106.8,101.5,101.5,108.3,101.9,102.1
2003-02-01,102.8,101.5,102.5,100.6,98.7,107.1,116.1,100.8,100.6,108.1,102.1,102.0,113.0,103.0,102.6
2003-03-01,103.1,101.7,103.2,100.6,100.3,107.6,117.2,100.5,100.3,108.3,102.3,102.2,115.3,103.6,102.7
2003-04-01,102.4,101.8,102.3,100.7,98.7,105.4,106.7,101.2,100.0,108.9,102.3,102.2,106.0,101.9,102.9
2003-05-01,102.5,102.1,102.9,100.6,98.3,104.0,99.3,101.4,100.7,110.2,102.5,102.4,103.7,101.8,103.2


In [22]:
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score

# Normalize the data
#data_norm = (data - data.mean()) / data.std()

# Split the data into training and testing sets
train_data = data.iloc[:200]
test_data = data.iloc[200:]

# Define the model architecture
input_seq = Input(shape=(None, 1))
encoder_lstm = LSTM(64, return_sequences=True, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(input_seq)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None, 1))
decoder_lstm = LSTM(64, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)

attention_layer = Dot(axes=[2, 2])([decoder_outputs, encoder_outputs])
attention_weights = Activation('softmax')(attention_layer)

context_vector = Dot(axes=[2, 1])([attention_weights, encoder_outputs])
decoder_combined_context = Concatenate(axis=-1)([context_vector, decoder_outputs])

output = TimeDistributed(Dense(1, activation='linear'))(decoder_combined_context)

model = Model([input_seq, decoder_inputs], output)
model.compile(optimizer='adam', loss='mse')

# Train the model
model.fit([train_data[['All-items']].values[:-1, :, np.newaxis], train_data[['All-items']].values[1:, :, np.newaxis]],
          train_data[['All-items']].values[1:, :, np.newaxis],
          epochs=200, batch_size=1)




Epoch 1/200
199/199 [==============================] - 2s 2ms/step - loss: 11503.9238
Epoch 2/200
199/199 [==============================] - 0s 2ms/step - loss: 7269.1870
Epoch 3/200
199/199 [==============================] - 0s 2ms/step - loss: 4809.8657
Epoch 4/200
199/199 [==============================] - 0s 2ms/step - loss: 3141.1172
Epoch 5/200
199/199 [==============================] - 0s 2ms/step - loss: 1998.4441
Epoch 6/200
199/199 [==============================] - 0s 2ms/step - loss: 1225.9352
Epoch 7/200
199/199 [==============================] - 0s 2ms/step - loss: 705.9592
Epoch 8/200
199/199 [==============================] - 0s 1ms/step - loss: 400.6733
Epoch 9/200
199/199 [==============================] - 0s 2ms/step - loss: 237.4167
Epoch 10/200
199/199 [==============================] - 0s 2ms/step - loss: 155.6402
Epoch 11/200
199/199 [==============================] - 0s 2ms/step - loss: 117.7961
Epoch 12/200
199/199 [==============================] - 0s 1ms/step

199/199 [==============================] - 0s 2ms/step - loss: 0.4218
Epoch 100/200
199/199 [==============================] - 0s 2ms/step - loss: 0.1989
Epoch 101/200
199/199 [==============================] - 0s 2ms/step - loss: 0.1620
Epoch 102/200
199/199 [==============================] - 0s 2ms/step - loss: 0.2175
Epoch 103/200
199/199 [==============================] - 0s 2ms/step - loss: 0.5045
Epoch 104/200
199/199 [==============================] - 0s 2ms/step - loss: 0.2788
Epoch 105/200
199/199 [==============================] - 0s 2ms/step - loss: 0.2632
Epoch 106/200
199/199 [==============================] - 0s 2ms/step - loss: 0.3442
Epoch 107/200
199/199 [==============================] - 0s 2ms/step - loss: 0.0950
Epoch 108/200
199/199 [==============================] - 0s 2ms/step - loss: 0.2059
Epoch 109/200
199/199 [==============================] - 0s 2ms/step - loss: 0.0579
Epoch 110/200
199/199 [==============================] - 0s 2ms/step - loss: 0.7501
Epoch 

199/199 [==============================] - 0s 2ms/step - loss: 0.1053
Epoch 197/200
199/199 [==============================] - 0s 2ms/step - loss: 0.3655
Epoch 198/200
199/199 [==============================] - 0s 2ms/step - loss: 0.1962
Epoch 199/200
199/199 [==============================] - 0s 2ms/step - loss: 0.0517
Epoch 200/200
199/199 [==============================] - 0s 2ms/step - loss: 1.0379


In [23]:

preds = model.predict([test_data[['All-items']].values[:-1, :, np.newaxis], test_data[['All-items']].values[:-1, :, np.newaxis]])

In [24]:

preds = model.predict([test_data[['All-items']].values[:-1, :, np.newaxis], test_data[['All-items']].values[:-1, :, np.newaxis]])

In [25]:
result = pd.DataFrame({'Actual': test_data['All-items'].values[:-1], 'Predicted': preds.flatten()})
result.index = test_data.index[1:]
result['RMSE'] = np.sqrt((result['Actual'] - result['Predicted'])**2)
result['MAPE'] = abs((result['Actual'] - result['Predicted']) / result['Actual']) * 100
result['R2'] = r2_score(result['Actual'], result['Predicted'])
result


,Actual,Predicted,RMSE,MAPE,R2
Date,,,,,
2019-10-01,136.2,135.606644,0.593356,0.435651,0.346585
2019-11-01,136.6,135.919540,0.680460,0.498140,0.346585
2019-12-01,136.4,135.763824,0.636176,0.466404,0.346585
2020-01-01,136.4,135.763824,0.636176,0.466404,0.346585
2020-02-01,136.8,136.073761,0.726239,0.530876,0.346585
2020-03-01,137.4,136.527267,0.872733,0.635177,0.346585
2020-04-01,136.6,135.919540,0.680460,0.498140,0.346585
2020-05-01,135.7,135.207397,0.492603,0.363009,0.346585
2020-06-01,136.1,135.527512,0.572488,0.420638,0.346585


##### 12 month forecast 

In [26]:

# Create a DataFrame with the last 15 months of the test data and the 3 months to forecast
last_15_months = test_data.iloc[-15:]
next_12_months = pd.date_range(start=test_data.index[-1], periods=12, freq='MS')
forecast_data = pd.DataFrame(index=next_12_months, columns=data.columns)

# Predict the CPI for the next 3 months
for i in range(12):
    input_data = last_15_months[['All-items']].values[np.newaxis, :, np.newaxis]
    prediction = model.predict([input_data, input_data])
    # Get the last predicted value only
    prediction = prediction[-1][-1][-1]
    forecast_data.iloc[i] = prediction
    # Update the input data with the predicted value for the next iteration
    last_15_months = last_15_months.append(forecast_data.iloc[i])

# Print the forecasted CPI for the next 3 months
print(forecast_data['All-items'])






2023-03-01    164.621811
2023-04-01    164.658875
2023-05-01    164.687729
2023-06-01    164.711533
2023-07-01    164.731506
2023-08-01    164.748337
2023-09-01    164.762726
2023-10-01    164.775116
2023-11-01    164.785828
2023-12-01    164.795197
2024-01-01    164.803436
2024-02-01     164.81073
Freq: MS, Name: All-items, dtype: object
